<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

### 加载Robot类
准备开始为JetBot编程前，我们需要导入“Robot”类。这个类允许我们轻松控制JetBot的电机

In [1]:
from jetbot import Robot

现在已经加载Robot类，我们可以用一下语句初始化这个instance（实例）

In [2]:
robot = Robot()

在本例程前我们已经试着转动过左右两边的电机,接下来我们使用更多关于控制Jetbot的方法

In [17]:
robot.down(0.5)#云台下降

In [16]:
robot.up(0.5)#云台上升

In [5]:
robot.vertical_motors_stop()#停止云台的上升下降运动

In [8]:
robot.forward(1)#Jetbot前进

In [9]:
robot.backward(1)#Jetbot后退

In [10]:
robot.left(0.75)#Jetbot左转

In [11]:
robot.right(0.75)#Jetbot右转

In [12]:
robot.stop()#停止Jetbot

In [13]:
robot.set_bln(1)#设置Jetbot的板载呼吸灯亮度为1(取值范围为 0 - 1.0)

有时可能我们只想在一段时间内移动机器人，为此，我们可以使用Python的time package。执行以下代码，加载time模块.

In [6]:
import time

In [9]:
robot.forward(0.6)
time.sleep(0.5)
robot.stop()

### 单独控制电机
上面我们看到了如何使用left，right等命令控制JetBot。但是如果我们想要单独设置每个电机速度怎么办？其实，有两种方法可以做到这一点。

第一种方法是调用set_motors方法。 例如，左转一秒，我们可以将左电机速度设置为30％，将右电机设置为60％，这将实现不同弧度的转向方式。

In [ ]:
robot.set_motors(0.5, 0.5)
time.sleep(1.0)
robot.stop()

In [8]:
robot.left_motor.value = 0.5
robot.right_motor.value = 0.9
time.sleep(1.0)
robot.left_motor.value = 0.0
robot.right_motor.value = 0.0

### 使用traitlets库连接到Jupyter lab HTML控件操作电机
接下来介绍一个非常酷的功能，就是在Jupyter Notbooks中可以让我们在这个页面上制作一些图形小按钮（控件），而使用traitlets可以连接这些小部件进行控制操作。这样，我们就可以通过网页的按钮，去控制我们的小车，这将会变得非常方便好玩。

为了说明如何编写程序，我们先创建并显示两个用于控制电机的滑块。

In [29]:
import ipywidgets.widgets as widgets
from IPython.display import display

# create two sliders with range [-1.0, 1.0]
left_slider = widgets.FloatSlider(description='left', min=-1.0, max=1.0, step=0.01, orientation='vertical')
right_slider = widgets.FloatSlider(description='right', min=-1.0, max=1.0, step=0.01, orientation='vertical')

# create a horizontal box container to place the sliders next to eachother
slider_container = widgets.HBox([left_slider, right_slider])

# display the container in this cell's output
display(slider_container)

你应该看见两个垂直的滑块显示在上面。

技巧提示：在Jupyter Lab，其实你可以把单元格弹出到其他窗口，例如这两个滑块。虽然不在同一窗口，但它仍然连接着这个notebook。具体操作是，鼠标移动到单元格（例如：滑块）上右键，选择「Creat new view for output」（为输出创建新窗口），然后拖动窗口到你满意的地方即可。

尝试单击并上下拖动滑块，会见到数值的变化。 请注意，当前我们移动滑块时JetBot的电机是没有任何反应的，那是因为我们还没有将它们连接到电机上！ 下面我们将通过使用traitlets包中的link函数来实现。

In [30]:
import traitlets
left_link = traitlets.link((left_slider, 'value'), (robot.left_motor, 'value'))
right_link = traitlets.link((right_slider, 'value'), (robot.right_motor, 'value'))

现在尝试拖动滑块（要先慢慢地拖动，以免你的JetBot突然冲出边界造成损坏），您应该看到相应的电机在转动！

我们上面创建的link函数实际上创建了一个双向链接！ 那意味着， 如果我们在其他地方设置电机值，滑块将会跟着更新！ 尝试执行下面的代码块：

In [31]:
robot.forward(0.5)
time.sleep(1.0)
robot.stop()

执行上面代码你应该看见滑块也发生了改变，响应了电机的速度值。如果我们要断开此连接，我们可以调用unlink方法逐一断开连接。

In [32]:
left_link.unlink()
right_link.unlink()

但是如果我们不想要一个双向的连接，比如说我们只想用滑块来显示电机的速度值，而不想用来控制，那么要实现这种功能，我们就可以使用dlink函数，左边是来源，右边是目标，（数据来源于电机，然后要显示在目标上）。 

In [33]:
left_link = traitlets.dlink((robot.left_motor, 'value'), (left_slider, 'value'))
right_link = traitlets.dlink((robot.right_motor, 'value'), (right_slider, 'value'))

### 将函数添加到事件
另一种使用traitlets的方法是把函数附加到事件中(例如 forward) 。只要对对象发生改变，就会调用函数，并将传递改变了的一些信息，例如old 值和new值。

先让我们创建一些用来控制机器人的按钮显示在notebook上。

In [34]:
# 创建按钮
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# 显示按钮
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

你应该看到上面显示的一组机器人控制按钮，但现在你点击按钮并不会有任何反应。要做到控制，我们需要创建一些函数附加到按钮on_click事件中

In [35]:
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.8)
    time.sleep(0.5)
    robot.stop()

def step_backward(change):
    robot.backward(0.8)
    time.sleep(0.5)
    robot.stop()

def step_left(change):
    robot.left(0.6)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.6)
    time.sleep(0.5)
    robot.stop()

现在我们已经定义了那些函数，让我们把这些函数附加到每一个按钮的on_click事件

In [36]:
# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

In [ ]:
执行完了上面的单元格后现在当你点击每一个按钮时，你应该看到JetBot都会对应作出移动。

#### 心跳开关
这里我们显示怎么去使用’heartbeat’ package 来停止JetBot的的移动。
这是检测JetBot与浏览器的连接是否还存在的简单方法。可以通过下面显
示的滑块调整心跳周期（以秒为单位），如果两次心跳之内不能在浏览器
之间往返通信的，那么心跳的’status‘ （状态）属性值将会设置为dead，
一旦连接恢复连接，status属性将设置为alive。

In [ ]:
from jetbot import Heartbeat

heartbeat = Heartbeat()

# this function will be called when heartbeat 'alive' status changes
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        robot.stop()
        
heartbeat.observe(handle_heartbeat_status, names='status')

period_slider = widgets.FloatSlider(description='period', min=0.001, max=0.5, step=0.01, value=0.5)
traitlets.dlink((period_slider, 'value'), (heartbeat, 'period'))

display(period_slider, heartbeat.pulseout)